In [39]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
from tqdm import tqdm
import random
from scipy import sparse
import numpy as np
import pandas as pd
from sklearn import preprocessing
import time

In [79]:
train_pd = pd.read_csv("./data/train.csv", encoding="utf-8")

In [80]:
word_list = []
vocab_list = []
for line in tqdm(train_pd.value):
    word_list.append([w for w in line])
    vocab_list.extend([w for w in line])

100%|█████████████████████████████████████████████████████████████████████████| 30000/30000 [00:00<00:00, 88001.13it/s]


In [81]:
c = Counter(vocab_list)
vocab = np.array(list(c.keys()))
vocab.sort()
vocab

array(['%', '&', '*', '+', '-', '.', '0', '1', '2', '3', '4', '5', '6',
       '7', '8', '9', '=', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
       'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
       'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
       'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
       'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [82]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(vocab)
label_encoder.classes_
label_encoder.transform(vocab)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69], dtype=int64)

In [85]:
word_labelencoder = [list(label_encoder.transform(w)) for w in word_list]
word_size = [len(i) for i in word_labelencoder]

In [86]:
train_size_pd = pd.DataFrame(word_size, columns=['len'])
train_size_pd['label'] = train_pd.label
train_size_pd.describe()

,len,label
count,30000.000000,30000.000000
mean,113.928533,0.083333
std,173.427420,0.276390
min,16.000000,0.000000
25%,53.000000,0.000000
50%,73.000000,0.000000
75%,100.000000,0.000000
max,11553.000000,1.000000


In [87]:
train_size_pd = train_size_pd[train_size_pd.len < 500]
Counter(train_size_pd.label)

Counter({0: 27159, 1: 1973})

In [105]:
pos_index = train_size_pd[train_size_pd.label == 1].index.values
random.shuffle(pos_index)
pos_index, pos_index.shape

(array([  110, 18753, 24250, ..., 26117, 28250, 12518], dtype=int64), (1973,))

In [106]:
neg_index = train_size_pd[train_size_pd.label == 0].index.values
neg_index = np.random.choice(neg_index, size=4000-1973)
neg_index, neg_index.shape

(array([12093,  4519, 28089, ..., 12443,  4628, 25168], dtype=int64), (2027,))

In [133]:
balance_train_index = np.append(pos_index,neg_index)
random.shuffle(balance_train_index)
balance_train_index, balance_train_index.shape

(array([ 8052,  1118, 12631, ..., 25999, 11073, 20257], dtype=int64), (4000,))

In [135]:
x_batch = np.array(word_labelencoder.copy())
x_batch = x_batch[balance_train_index]
y_batch = np.array(train_pd.label.copy())
y_batch = y_batch[balance_train_index]
x_batch_size = [len(i) for i in x_batch]
x_batch.shape, y_batch.shape, len(x_batch_size)

((4000,), (4000,), 4000)

In [138]:
y_batch[:100]

array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1], dtype=int64)

In [139]:
max_size = np.max(x_batch_size)
max_size

499

In [140]:
x_batch_pad = []
for x in tqdm(x_batch[:]):
    list_test = list()
    list_test =([-1] * max_size)
    list_test[:len(x)] = x
    list_test = [[i] for i in list_test]
    x_batch_pad.append(list_test)

100%|███████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 17497.32it/s]


In [46]:
x_batch = np.array(word_labelencoder.copy())
y_batch = np.array(train_pd.label.copy())
x_batch_size = [len(i) for i in x_batch]
# len(x), len(y)
x_batch.shape, y_batch.shape, len(x_batch_size)

((100,), (100,), 100)

In [47]:
max_size = np.max(x_batch_size)
max_size

823

In [52]:
x_batch_pad = []
for x in tqdm(x_batch[:]):
    list_test = list()
    list_test =([-1] * max_size)
    list_test[:len(x)] = x
    list_test = [[i] for i in list_test]
    x_batch_pad.append(list_test)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 11082.56it/s]


In [141]:
x_batch_pad[:10]

[[[54],
  [48],
  [68],
  [58],
  [57],
  [48],
  [16],
  [7],
  [6],
  [6],
  [13],
  [5],
  [7],
  [8],
  [14],
  [6],
  [8],
  [5],
  [10],
  [14],
  [10],
  [12],
  [11],
  [5],
  [7],
  [6],
  [6],
  [8],
  [6],
  [6],
  [9],
  [6],
  [6],
  [6],
  [6],
  [6],
  [6],
  [6],
  [6],
  [1],
  [54],
  [48],
  [68],
  [63],
  [66],
  [58],
  [16],
  [7],
  [7],
  [10],
  [13],
  [3],
  [64],
  [57],
  [52],
  [58],
  [57],
  [3],
  [62],
  [48],
  [55],
  [48],
  [46],
  [63],
  [3],
  [7],
  [0],
  [8],
  [19],
  [8],
  [0],
  [8],
  [19],
  [9],
  [0],
  [8],
  [19],
  [10],
  [0],
  [8],
  [19],
  [46],
  [58],
  [57],
  [46],
  [44],
  [63],
  [43],
  [66],
  [62],
  [0],
  [8],
  [14],
  [6],
  [67],
  [9],
  [44],
  [0],
  [8],
  [19],
  [64],
  [62],
  [48],
  [61],
  [0],
  [8],
  [14],
  [0],
  [8],
  [15],
  [0],
  [8],
  [19],
  [47],
  [44],
  [63],
  [44],
  [45],
  [44],
  [62],
  [48],
  [0],
  [8],
  [14],
  [0],
  [8],
  [15],
  [0],
  [8],
  [19],
  [65],
  [48],
  [6

In [142]:
y_batch_pad = [np.eye(2)[item] for item in y_batch]
y_batch_pad = [list(i) for i in y_batch_pad]

In [143]:
y_batch_pad

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],

In [144]:
x_batch_size

[214,
 253,
 331,
 443,
 103,
 156,
 22,
 98,
 283,
 142,
 106,
 235,
 372,
 241,
 81,
 75,
 403,
 208,
 296,
 140,
 319,
 199,
 26,
 79,
 336,
 156,
 184,
 64,
 69,
 90,
 81,
 444,
 150,
 110,
 318,
 175,
 196,
 75,
 177,
 68,
 59,
 145,
 88,
 211,
 192,
 362,
 261,
 128,
 35,
 143,
 290,
 344,
 41,
 62,
 252,
 92,
 319,
 71,
 280,
 181,
 145,
 249,
 155,
 78,
 105,
 42,
 96,
 186,
 128,
 360,
 144,
 61,
 224,
 81,
 452,
 81,
 204,
 124,
 110,
 274,
 222,
 35,
 88,
 455,
 394,
 32,
 260,
 238,
 187,
 118,
 414,
 126,
 267,
 463,
 405,
 83,
 113,
 62,
 72,
 162,
 56,
 467,
 382,
 414,
 58,
 40,
 184,
 94,
 58,
 136,
 35,
 203,
 50,
 226,
 88,
 165,
 100,
 81,
 149,
 262,
 73,
 199,
 73,
 84,
 418,
 105,
 434,
 124,
 129,
 335,
 36,
 309,
 84,
 192,
 33,
 477,
 196,
 62,
 85,
 160,
 86,
 481,
 62,
 388,
 337,
 88,
 30,
 463,
 175,
 35,
 60,
 39,
 94,
 346,
 200,
 95,
 399,
 48,
 110,
 60,
 83,
 60,
 188,
 63,
 102,
 85,
 407,
 450,
 72,
 113,
 144,
 243,
 187,
 94,
 346,
 112,
 181,
 31

In [145]:
class Batcher():
    def __init__(self, x, y, x_batch_size):
        self.split_size = int(len(x)*0.8)
        self.train_x = x[:self.split_size]
        self.train_y = y[:self.split_size]
        self.train_size = x_batch_size[:self.split_size]
        self.test_x = x[self.split_size:]
        self.test_y = y[self.split_size:]
        self.test_size = x_batch_size[self.split_size:]
        self.start = 0
    def next_batch(self, batch_size):
        s_index = self.start
        e_index = self.start + batch_size
        if e_index >= self.split_size:
            self.start = 0
            s_index = self.start
            e_index = self.start + batch_size
        self.start = e_index
        return self.train_x[s_index:e_index], self.train_y[s_index:e_index], self.train_size[s_index:e_index]

In [146]:
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

In [147]:
batch_data, batch_labels, batch_seqlen = sqli_batch.next_batch(10)

In [148]:
batch_data

[[[54],
  [48],
  [68],
  [58],
  [57],
  [48],
  [16],
  [7],
  [6],
  [6],
  [13],
  [5],
  [7],
  [8],
  [14],
  [6],
  [8],
  [5],
  [10],
  [14],
  [10],
  [12],
  [11],
  [5],
  [7],
  [6],
  [6],
  [8],
  [6],
  [6],
  [9],
  [6],
  [6],
  [6],
  [6],
  [6],
  [6],
  [6],
  [6],
  [1],
  [54],
  [48],
  [68],
  [63],
  [66],
  [58],
  [16],
  [7],
  [7],
  [10],
  [13],
  [3],
  [64],
  [57],
  [52],
  [58],
  [57],
  [3],
  [62],
  [48],
  [55],
  [48],
  [46],
  [63],
  [3],
  [7],
  [0],
  [8],
  [19],
  [8],
  [0],
  [8],
  [19],
  [9],
  [0],
  [8],
  [19],
  [10],
  [0],
  [8],
  [19],
  [46],
  [58],
  [57],
  [46],
  [44],
  [63],
  [43],
  [66],
  [62],
  [0],
  [8],
  [14],
  [6],
  [67],
  [9],
  [44],
  [0],
  [8],
  [19],
  [64],
  [62],
  [48],
  [61],
  [0],
  [8],
  [14],
  [0],
  [8],
  [15],
  [0],
  [8],
  [19],
  [47],
  [44],
  [63],
  [44],
  [45],
  [44],
  [62],
  [48],
  [0],
  [8],
  [14],
  [0],
  [8],
  [15],
  [0],
  [8],
  [19],
  [65],
  [48],
  [6

In [149]:
batch_labels

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0]]

In [150]:
batch_seqlen

[214, 253, 331, 443, 103, 156, 22, 98, 283, 142]

In [ ]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_steps = 10000
batch_size = 100
display_step = 100

# Network Parameters
seq_max_len = max_size # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

# trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
# testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)
sqli_batch = Batcher(x_batch_pad, y_batch_pad, x_batch_size)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 1])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps + 1):
#         batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        batch_x, batch_y, batch_seqlen = sqli_batch.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
#             print("Step " + str(step*batch_size) + ", Minibatch Loss= " + \
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
#     test_data = testset.data
#     test_label = testset.labels
#     test_seqlen = testset.seqlen
    test_data = sqli_batch.test_x
    test_label = sqli_batch.test_y
    test_seqlen = sqli_batch.test_size
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Step 100, Minibatch Loss= 4.254035, Training Accuracy= 0.61000
Step 10000, Minibatch Loss= 0.599294, Training Accuracy= 0.66000
Step 20000, Minibatch Loss= 0.650070, Training Accuracy= 0.60000
Step 30000, Minibatch Loss= 0.733956, Training Accuracy= 0.58000
Step 40000, Minibatch Loss= 0.246936, Training Accuracy= 0.96000
Step 50000, Minibatch Loss= 0.115856, Training Accuracy= 0.99000
Step 60000, Minibatch Loss= 0.063737, Training Accuracy= 1.00000
Step 70000, Minibatch Loss= 0.138618, Training Accuracy= 0.98000
Step 80000, Minibatch Loss= 0.146792, Training Accuracy= 0.97000
Step 90000, Minibatch Loss= 0.121727, Training Accuracy= 0.97000
Step 100000, Minibatch Loss= 0.090169, Training Accuracy= 0.97000
Step 110000, Minibatch Loss= 0.058737, Training Accuracy= 0.97000
Step 120000, Minibatch Loss= 0.072674, Training Accuracy= 0.98000
Step 130000, Minibatch Loss= 0.112341, Training Accuracy= 0.97000
Step 140000, Minibatch Loss= 0.108819, Training Accuracy= 0.97000
Step 150000, Minibatch

In [ ]:
""

In [18]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

# 创建输入数据
X = np.random.randn(2, 10, 8)

# 第二个example长度为6
X[1,6:] = 0
X_lengths = [10, 6]

cell = tf.contrib.rnn.BasicLSTMCell(num_units=64, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

print(result[0])

assert result[0]["outputs"].shape == (2, 10, 64)

# 第二个example中的outputs超过6步(7-10步)的值应该为0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
{'outputs': array([[[ 0.04973484,  0.06838848, -0.00098619, ...,  0.03823282,
         -0.02151134,  0.00838239],
        [ 0.01663709,  0.09550603, -0.04364772, ..., -0.02928781,
         -0.0102941 ,  0.07940422],
        [ 0.07598038,  0.12220079, -0.11426227, ..., -0.07287159,
          0.02567821,  0.08098095],
        ..., 
        [ 0.1765959 ,  0.02437624,  0.07324075, ...,  0.00578768,
          0.06607261, -0.05371523],
        [ 0.17451524,  0.05487746,  0.05294119, ..., -0.00266275,
          0.09300841,  0.00595578],
        

In [26]:
from __future__ import print_function

import tensorflow as tf
import random


# ====================
#  TOY DATA GENERATOR
# ====================
class ToySequenceData(object):
    """ Generate sequence of data with dynamic length.
    This class generate samples for training:
    - Class 0: linear sequences (i.e. [0, 1, 2, 3,...])
    - Class 1: random sequences (i.e. [1, 3, 10, 7,...])
    NOTICE:
    We have to pad each sequence to reach 'max_seq_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform thanks to
    'seqlen' attribute that records every actual sequence length.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Random sequence length
            len = random.randint(min_seq_len, max_seq_len)
            # Monitor sequence length for TensorFlow dynamic calculation
            self.seqlen.append(len)
            # Add a random or linear int sequence (50% prob)
            if random.random() < .5:
                # Generate a linear sequence
                rand_start = random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generate a random sequence
                s = [[float(random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [30]:
trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
batch_data, batch_labels, batch_seqlen = trainset.next(100)

In [31]:
batch_data

[[[0.927],
  [0.928],
  [0.929],
  [0.93],
  [0.931],
  [0.932],
  [0.933],
  [0.934],
  [0.935],
  [0.936],
  [0.937],
  [0.938],
  [0.939],
  [0.94],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0]],
 [[0.292],
  [0.293],
  [0.294],
  [0.295],
  [0.296],
  [0.297],
  [0.298],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0]],
 [[0.397],
  [0.069],
  [0.933],
  [0.131],
  [0.876],
  [0.817],
  [0.119],
  [0.59],
  [0.691],
  [0.174],
  [0.346],
  [0.528],
  [0.346],
  [0.639],
  [0.721],
  [0.89],
  [0.667],
  [0.108],
  [0.084],
  [0.985]],
 [[0.212],
  [0.213],
  [0.214],
  [0.215],
  [0.216],
  [0.217],
  [0.218],
  [0.219],
  [0.22],
  [0.221],
  [0.222],
  [0.223],
  [0.224],
  [0.225],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0],
  [0.0]],
 [[0.737],
  [0.738],
  [0.739],
  [0.74],
  [0.741],
  [0.742],
  [0.743],
  [0.744],
  [0.745],
  [0.746],
  [0.747],
  [0.748],
  [0.749],
  [0.75],
  [0.751],
  [0.752

In [32]:
batch_labels

[[1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [1.0, 0.0],
 [0.0, 1.0],
 [1.0, 0.0],

In [33]:
batch_seqlen

[14,
 7,
 20,
 14,
 17,
 6,
 8,
 7,
 17,
 20,
 14,
 18,
 8,
 16,
 15,
 7,
 11,
 8,
 15,
 13,
 8,
 9,
 10,
 14,
 12,
 10,
 16,
 15,
 15,
 4,
 17,
 19,
 9,
 11,
 16,
 4,
 6,
 5,
 12,
 13,
 9,
 18,
 16,
 15,
 13,
 19,
 3,
 18,
 4,
 9,
 10,
 18,
 16,
 19,
 16,
 7,
 9,
 19,
 6,
 8,
 12,
 19,
 6,
 4,
 4,
 17,
 14,
 3,
 14,
 8,
 10,
 11,
 20,
 8,
 20,
 11,
 10,
 10,
 14,
 13,
 15,
 17,
 17,
 12,
 16,
 15,
 16,
 4,
 19,
 8,
 6,
 15,
 7,
 9,
 17,
 6,
 5,
 12,
 13,
 3]

In [25]:
tf.reset_default_graph()
""" Dynamic Recurrent Neural Network.
TensorFlow implementation of a Recurrent Neural Network (LSTM) that performs
dynamic computation over sequences with variable length. This example is using
a toy dataset to classify linear sequences. The generated sequences have
variable length.
Links:
    [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""




# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.01
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parameters
seq_max_len = 20 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

trainset = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
testset = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 1])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def dynamicRNN(x, seqlen, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps + 1):
        batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Step 128, Minibatch Loss= 0.747308, Training Accuracy= 0.32812
Step 25600, Minibatch Loss= 0.697757, Training Accuracy= 0.45192
Step 51200, Minibatch Loss= 0.695742, Training Accuracy= 0.44231
Step 76800, Minibatch Loss= 0.692827, Training Accuracy= 0.45192
Step 102400, Minibatch Loss= 0.688300, Training Accuracy= 0.49038
Step 128000, Minibatch Loss= 0.679706, Training Accuracy= 0.63462
Step 153600, Minibatch Loss= 0.659868, Training Accuracy= 0.72115
Step 179200, Minibatch Loss= 0.601811, Training Accuracy= 0.75000
Step 204800, Minibatch Loss= 0.486323, Training Accuracy= 0.79808
Step 230400, Minibatch Loss= 0.414720, Training Accuracy= 0.80769
Step 256000, Minibatch Loss= 0.392889, Training Accuracy= 0.81731
Step 281600, Minibatch Loss= 0.384932, Training Accuracy= 0.81731
Step 307200, Minibatch Loss= 0.380648, Training Accuracy= 0.81731
Step 332800, Minibatch Loss= 0.377638, Training Accuracy= 0.81731
Step 358400, Minibatch Loss= 0.375261, Training Accuracy= 0.81731
Step 384000, Min

KeyboardInterrupt: 

In [ ]:
test_data = testset.data
test_label = testset.labels
test_seqlen = testset.seqlen
print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))